# Homework 4

## v0.2

In [ ]:
import numpy as np
import mujoco
import mujoco.viewer as viewer
import modern_robotics as mr

xml="""<mujoco model="3R_spatial">
    <option gravity="0 0 0"/>
    <compiler angle="degree"/>
    <asset>  
        <texture name="grid" type="2d" builtin="checker" rgb1=".1 .2 .3"
                 rgb2=".2 .3 .4" width="300" height="300" mark="none"/>
        <material name="grid" texture="grid" texrepeat="6 6" texuniform="true" reflectance=".2"/>
    </asset>
    <default>
        <joint type="hinge" axis="0 0 1" limited="true"/>
        <geom type="cylinder" size=".025 .2" pos="0 0 0"/>
    </default>

    <worldbody>
        <light diffuse=".5 .5 .5" pos="0 0 3" dir="0 0 -1"/>
        <geom type="plane" size="1 1 0.1" material="grid"/>  
        <body name="link1" pos="0 0 .3">
            <joint name="joint1" pos="0 0 0" range="-180 180" axis="0 0 1" damping="0.1"/>
            <geom pos="0 0 0" rgba=".6 .2 .2 1" size=".0125 .3"/>
            <geom pos="0 0 0" rgba=".6 .2 .2 1" size=".025 .1"/>
            <geom name="fix1" pos=".1 0 0" euler="0 90 0" size=".0125 .1" rgba=".2 .6 1 1"/>
            <body name="link2" pos=".2 0 0" euler="90 0 -90">
                <joint name="joint2" pos="0 0 0" range="0 180" axis="0 0 1" damping="0.1"/>
                <geom pos="0 0 0" rgba=".6 .2 .2 1" size=".0125 .3"/>
                <geom pos="0 0 0" rgba=".6 .2 .2 1" size=".025 .1"/>
                <geom name="fix2" pos=".1 0 0" euler="0 90 0" size=".0125 .1" rgba=".2 .6 1 1"/>
                <body name="link3" pos=".2 0 0" euler="-90 0 0">
                    <joint name="joint3" pos="0 0 0" range="-90 90" axis="0 0 1" damping="0.1"/>
                    <geom pos="0 0 0" rgba=".6 .2 .2 1" size=".0125 .15"/>
                    <geom pos="0 0 0" rgba=".6 .2 .2 1" size=".025 .05"/>
                    <site name="end_effector" pos="0 0 0" size="0.001" rgba="0 1 0 1"/>
                </body>
            </body>
        </body>
    </worldbody>


    <actuator>
        <position name="position_control1" joint="joint1" kp=".8" kv="0.02" ctrlrange="-3.1416 3.1416"/>
        <position name="position_control2" joint="joint2" kp=".8" kv="0.02" ctrlrange="0 3.1416"/>
        <position name="position_control3" joint="joint3" kp=".8" kv="0.02" ctrlrange="-1.5708 1.5708"/>
    </actuator>
    
    <sensor>
        <jointpos name="joint1_position_sensor" joint="joint1"/>
        <jointpos name="joint2_position_sensor" joint="joint2"/>
        <jointpos name="joint3_position_sensor" joint="joint3"/>
    </sensor>
    


</mujoco>"""


# load
model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)
# mujoco.mj_resetDataKeyframe(model, data, 0)
# viewer.launch(model,data)

In [3]:
# 向量化 NearZero 函数并应用于数组
NearZero_vec = np.vectorize(mr.NearZero)

In [4]:
# 获取链接位置和旋转矩阵的函数
def get_link_pose(model, data, link_name):
    try:
        link_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_BODY, link_name)
        position = data.xpos[link_id]
        rotation_matrix = data.xmat[link_id].reshape(3, 3)
        print(f"{link_name} 位置（position）:", position)
        print(f"{link_name} 旋转矩阵（rotation_matrix）:\n", rotation_matrix, "\n")
        return position, rotation_matrix
    except Exception as e:
        print(f"Error accessing {link_name}:", e)
        return None, None

In [5]:
# 定义正向运动学的初始参数
M = np.array([[0, 0, 1, 0.2],
              [0, 1, 0, 0],
              [-1, 0, 0, 0.1],
              [0, 0, 0, 1]])
Blist = np.array([[ 0, 0, 0, 0, 0, 1],
                  [0.3, 0, -0.2, 0, -1, 0,],
                  [ 0, 0.1, 0 ,1, 0, 0]]).T

In [10]:
mujoco.mj_resetDataKeyframe(model, data, 0)
viewer.launch(model,data)

In [170]:
# mujoco.mj_step(model, data)
np.set_printoptions(suppress=True, precision=3)
# 打印 link1, link2, link3 的位置信息和旋转矩阵
get_link_pose(model, data, "link1")
get_link_pose(model, data, "link2")
get_link_pose(model, data, "link3")

link1 位置（position）: [0.  0.  0.3]
link1 旋转矩阵（rotation_matrix）:
 [[ 0.79  -0.613  0.   ]
 [ 0.613  0.79   0.   ]
 [ 0.     0.     1.   ]] 

link2 位置（position）: [0.158 0.123 0.3  ]
link2 旋转矩阵（rotation_matrix）:
 [[ 0.     0.79   0.613]
 [ 0.     0.613 -0.79 ]
 [-1.     0.     0.   ]] 

link3 位置（position）: [0.158 0.123 0.1  ]
link3 旋转矩阵（rotation_matrix）:
 [[ 0.    -0.613  0.79 ]
 [ 0.     0.79   0.613]
 [-1.     0.     0.   ]] 



(array([0.158, 0.123, 0.1  ]),
 array([[ 0.   , -0.613,  0.79 ],
        [ 0.   ,  0.79 ,  0.613],
        [-1.   ,  0.   ,  0.   ]]))

In [133]:
# jiont1
M1 = np.array([[1, 0, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 1, 0.3],
              [0, 0, 0, 1]])
Blist1 = np.array([[0, 0, 1, 0, 0, 0]]).T
theta1 = data.qpos[:1]
print("theta1:",theta1,'\n')
mr.FKinBody(M1, Blist1, theta1)

theta1: [0.817] 



array([[ 0.684, -0.729,  0.   ,  0.   ],
       [ 0.729,  0.684,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  1.   ,  0.3  ],
       [ 0.   ,  0.   ,  0.   ,  1.   ]])

In [138]:
# jiont2
M2 = np.array([[0, 1, 0, 0.2],
              [0, 0, -1, 0],
              [-1, 0, 0, 0.3],
              [0, 0, 0, 1]])
Blist2 = np.array([[0, 0, 1, 0, 0, 0],
                   [0, -1, 0, 0.3, 0, -0.2]]).T
theta2 = data.qpos[:2] 
print("theta2:",theta2,'\n')
mr.FKinBody(M2, Blist2, theta2)

theta2: [0.817 0.   ] 



array([[ 0.729,  0.684,  0.   ,  0.2  ],
       [ 0.   ,  0.   , -1.   ,  0.   ],
       [-0.684,  0.729,  0.   ,  0.3  ],
       [ 0.   ,  0.   ,  0.   ,  1.   ]])

In [171]:
# 获取并打印末端执行器的位置和旋转矩阵
try:
    site_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_SITE, "end_effector")
    position = data.site_xpos[site_id]
    rotation_matrix = data.site_xmat[site_id].reshape(3, 3)
    print("末端执行器位置（position）:", position)
    print("末端执行器旋转矩阵（rotation_matrix）:\n", rotation_matrix)
except Exception as e:
    print("Error accessing end_effector:", e)

# 从 MuJoCo 获取关节角度并计算正向运动学
thetalist = data.qpos[:3]
print("\ntheta:", thetalist)
print("\nKinematic position of end-effector in base frame: ")
print(mr.FKinSpace(M, Blist, thetalist))

末端执行器位置（position）: [0.158 0.123 0.1  ]
末端执行器旋转矩阵（rotation_matrix）:
 [[ 0.    -0.613  0.79 ]
 [ 0.     0.79   0.613]
 [-1.     0.     0.   ]]

theta: [0.66 0.   0.  ]

Kinematic position of end-effector in base frame: 
[[ 0.    0.    1.    0.2 ]
 [ 0.    1.    0.   -0.  ]
 [-1.    0.    0.    0.76]
 [ 0.    0.    0.    1.  ]]


In [176]:
T = np.array(M)
print("M",T,'\n')
for i in range(len(thetalist)):
    T = np.dot(mr.MatrixExp6(mr.VecTose3(np.array(Blist)[:, i] \
                                        * thetalist[i])),T)
T

M [[ 0.   0.   1.   0.2]
 [ 0.   1.   0.   0. ]
 [-1.   0.   0.   0.1]
 [ 0.   0.   0.   1. ]] 



array([[ 0.  ,  0.  ,  1.  ,  0.2 ],
       [ 0.  ,  1.  ,  0.  , -0.  ],
       [-1.  ,  0.  ,  0.  ,  0.76],
       [ 0.  ,  0.  ,  0.  ,  1.  ]])

In [ ]:
omg = mr.VecToso3([1, 0, 0])
p = np.array([0.2, 0, 0.1])
omg@p

In [148]:
import math
class Screw:

    # 尺寸与方向
    j1_axis = [0, 0, 1]
    j1_xyz = [0, 0, 0.3]
    j2_axis = [0, -1, 0]
    j2_xyz = [0.2, 0, 0.3]
    j3_axis = [1, 0, 0]
    j3_xyz = [0.2, 0, 0.1]

    # 单位速度螺旋
    j1_screw = np.array([np.cross(j1_xyz, j1_axis), j1_axis])
    j2_screw = np.array([np.cross(j2_xyz, j2_axis), j2_axis])
    j3_screw = np.array([np.cross(j3_xyz, j3_axis), j3_axis])

    # 末端起始位置
    ee_init = np.array([[0, 0, 1, 0.2],
                        [0, 1, 0, 0],
                        [-1, 0, 0, 0.1],
                        [0, 0, 0, 1]])


    def from_screw_to_skew(self,screw):#反对称矩阵
        axis = screw[1]
        skew = [[0, -axis[2], axis[1]],
                [axis[2], 0, -axis[0]],
                [-axis[1], axis[0], 0]]
        return skew

    def from_screw_to_rotation_matrix(self,theta,screw):#选转矩阵
        skew = np.array(self.from_screw_to_skew(screw))
        rotation = np.eye(3) + math.sin(theta) * skew + (1 - math.cos(theta)) * skew @ skew
        return rotation

    def from_screw_to_translation_vector(self,theta,screw):#平移向量
        skew = np.array(self.from_screw_to_skew(screw))
        xyz_T = np.array([screw[0]]).T
        axis = np.array([screw[1]])
        axis_T = np.array([screw[1]]).T
        rotation = np.array(self.from_screw_to_rotation_matrix(theta,screw))
        translation = (np.eye(3) - rotation) @ skew @ xyz_T + theta * axis @ xyz_T * axis_T
        return translation

    def from_screw_to_homogeneous_transformation_matrix(self,theta,screw):#齐次变换矩阵
        rotation = np.array(self.from_screw_to_rotation_matrix(theta, screw))
        translation = np.array(self.from_screw_to_translation_vector(theta,screw))
        # homogeneous 形式为 [[rotation,translation],
        #                    [0,0,0,1]]
        # 矩阵大小4*4
        rotation_with_translation = np.hstack((rotation,translation))#水平堆叠
        homogeneous = np.vstack((rotation_with_translation,np.array([0, 0, 0, 1])))#竖直堆叠
        return homogeneous

    def calculate_H_wrist(self,theta1,theta2,theta3):
        H_wrist = self.from_screw_to_homogeneous_transformation_matrix(theta1, self.j1_screw) \
                  @ self.from_screw_to_homogeneous_transformation_matrix(theta2, self.j2_screw) \
                  @ self.from_screw_to_homogeneous_transformation_matrix(theta3, self.j3_screw)
        return H_wrist

    def calculate_H_wrist_end(self,theta1,theta2,theta3):
        H_wrist_end = self.from_screw_to_homogeneous_transformation_matrix(theta1, self.j1_screw) \
                      @ self.from_screw_to_homogeneous_transformation_matrix(theta2, self.j2_screw) \
                      @ self.from_screw_to_homogeneous_transformation_matrix(theta3, self.j3_screw) @ self.ee_init
        return H_wrist_end


In [172]:
theta1 = data.qpos[:1]
theta2 = data.qpos[1:2] 
theta3 = data.qpos[2:3]
screw = Screw()
screw.calculate_H_wrist_end(theta1, theta2, theta3)

/tmp/ipykernel_9848/3924151297.py:33: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  rotation = np.eye(3) + math.sin(theta) * skew + (1 - math.cos(theta)) * skew @ skew


array([[ 0.   , -0.613,  0.79 ,  0.158],
       [ 0.   ,  0.79 ,  0.613,  0.123],
       [-1.   ,  0.   ,  0.   ,  0.1  ],
       [ 0.   ,  0.   ,  0.   ,  1.   ]])

In [161]:
# 尺寸与方向
j1_axis = [0, 0, 1]
j1_xyz = [0, 0, 0.3]
j2_axis = [0, -1, 0]
j2_xyz = [0.2, 0, 0.3]
j3_axis = [1, 0, 0]
j3_xyz = [0.2, 0, 0.1]

# 单位速度螺旋
j1_screw = np.array([np.cross(j1_xyz, j1_axis), j1_axis])
j2_screw = np.array([np.cross(j2_xyz, j2_axis), j2_axis])
j3_screw = np.array([np.cross(j3_xyz, j3_axis), j3_axis])
j1_screw
j2_screw
j3_screw

array([[0. , 0.1, 0. ],
       [1. , 0. , 0. ]])